In [2]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle

In [3]:
bw.projects.set_current('iw_integration')

# Save and load functions with pickle

Save function

In [4]:
def pickle_save(file_path_root,object_name_to_save,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    pickle.dump( object_name_to_save, open( complete_file_path, "wb" ) )

In [ ]:
#file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
#object_name_to_save=MC_results_eiv33_simplified_it1000_LCI0_LCIA1
#file_name='MC_results_eiv33_simplified_it1000_LCI0_LCIA1'

pickle_save(file_path_root,object_name_to_save,file_name)

Load function

In [4]:
def pickle_load(file_path_root,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    return pickle.load( open( complete_file_path, "rb" ) );

In [ ]:
#file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
#object_name_to_save=MC_results_eiv33_simplified_it1000_LCI0_LCIA1
#file_name='MC_results_eiv33_simplified_it1000_LCI0_LCIA1'

MC_results_eiv33_simplified_it1000_LCI0_LCIA1_loadtest=pickle_load(file_path_root,file_name)

# Saving IW+ impact methods

In [7]:
IW_with_uncert_beta=[m for m in bw.methods 
                     if 'IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017' in str(m)
                    and 'with uncert' in str(m)]
IW_with_uncert_beta_with_errors=[m for m in IW_with_uncert_beta 
                             if 'Land transformation' in str(m) 
                             or 'Land occupation' in str(m)
                             or 'Freshwater eutrophication' in str(m)]
IW_with_uncert_beta_with_errors_notID14=[m for m in IW_with_uncert_beta 
                             if ('Land transformation' in str(m) 
                             or 'Land occupation' in str(m)
                             or 'Freshwater eutrophication' in str(m)) and 'ID14' not in str(m)]
IW_with_uncert_beta_with_errors_ID14=[m for m in IW_with_uncert_beta 
                             if ('Land transformation' in str(m) 
                             or 'Land occupation' in str(m)
                             or 'Freshwater eutrophication' in str(m)) and 'ID14' in str(m)]


IW_with_uncert_beta_with_errors_ID14

[('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater eutrophication, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert ID14')]

In [13]:
#Change a CF value for uncertainty dictionary or other value

def modify_CF_value(CF_tuple, new_CF_value):
    
    lst=list(CF_tuple)
    lst[1]=new_CF_value #value or uncertainty dictionary
    new_CF_tuple=tuple(lst)
    
    return new_CF_tuple;


#Remove CF uncertainty for several impact categories

def modify_uncert_ID14_in_impact_cat(ic_name):

    ic=bw.Method(ic_name)
    cf_list=ic.load()
    
    i=0

    for cf in cf_list:

        if type(cf[1])== dict:
            cf[1]['uncertainty type']=14
            cf[1]['uncertainty_type']=14
            cf=modify_CF_value(cf, cf[1])

        cf_list[i]=cf
        i=i+1

    ic.write(cf_list)
        
    return;


#Copy and rename impact category

def copy_rename_impact_cat(old_ic_name, new_ic_name):
    
    old_ic=bw.Method(old_ic_name)
    old_ic.copy(new_ic_name)
    
    return;

In [22]:
list_of_ic_names=IW_with_uncert_beta_with_errors
string_to_replace='with uncert'
string_to_use_instead='with uncert ID14'

for ic_name in list_of_ic_names:
    
    #define new ic name. Here we changed only the first name
    new_ic_name=(ic_name[0],ic_name[1],ic_name[2].replace(string_to_replace,string_to_use_instead))
    
    #copy ic
    old_ic_name=ic_name
    copy_rename_impact_cat(old_ic_name, new_ic_name)
    
    #remove CF uncertainty
    modify_uncert_ID14_in_impact_cat(new_ic_name)

In [8]:
IC_ID14=[m for m in bw.methods if 'ID14' in str(m)]
    

In [9]:
IC_ID14

[('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater eutrophication, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert ID14')]

In [14]:
list_of_ic_names=IC_ID14
string_to_replace='- only with spatial variability - Four param beta integration '
string_to_use_instead=''

for ic_name in list_of_ic_names:
    
    #define new ic name. Here we changed only the first name
    new_ic_name=(ic_name[0].replace(string_to_replace,string_to_use_instead),ic_name[1],ic_name[2])
    
    #copy ic
    old_ic_name=ic_name
    copy_rename_impact_cat(old_ic_name, new_ic_name)

In [15]:
IC_ID14_short_names=[m for m in bw.methods if 'ID14' in str(m) and '- only with spatial variability - Four param beta integration ' not in str(m) ]

In [16]:
IC_ID14_short_names

[('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert ID14'),
 ('IMPACTWorld+ - Endpoint - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater eutrophication, GLO, with uncert ID14')]

In [15]:
bw.Method(('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert ID14')).load()

[(('biosphere3', '99127c79-69f5-4046-a298-3f842c68d62b'),
  {'amount': -168.8913863,
   'loc': 1.0337e-14,
   'maximum': -166.2035984,
   'minimum': -183.7061033,
   'shape': 1.87542e-15,
   'uncertainty type': 14,
   'uncertainty_type': 14}),
 (('biosphere3', 'e97b784a-ec09-4b1b-9f14-cc0ce9799c9e'),
  {'amount': 159.166675,
   'loc': 0.057658156,
   'maximum': 315.786837,
   'minimum': 32.06385025,
   'shape': 0.071048222,
   'uncertainty type': 14,
   'uncertainty_type': 14}),
 (('biosphere3', '00d15dd1-8517-49dd-933d-e7db77e9d342'),
  {'amount': -22.63665981,
   'loc': 5.06805e-16,
   'maximum': -8.310179922,
   'minimum': -41.67505986,
   'shape': 3.81373e-16,
   'uncertainty type': 14,
   'uncertainty_type': 14}),
 (('biosphere3', '99801a43-d476-4055-8d97-fb7c4ae173d4'),
  {'amount': -37.34469997,
   'loc': 0.27636933,
   'maximum': -0.493290004,
   'minimum': -91.41197914,
   'shape': 0.188369003,
   'uncertainty type': 14,
   'uncertainty_type': 14}),
 (('biosphere3', '4e1715ba-

In [53]:
ic_with_errors_ID14={}

for ic_name in IC_ID14:
    ic_with_errors_ID14[ic_name]=bw.Method(ic_name).load()

file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=ic_with_errors_ID14
file_name='ic_with_errors_ID14'

pickle_save(file_path_root,object_name_to_save,file_name)
    

Test exporting object with built-in function in BW2 : https://docs.brightwaylca.org/technical/bw2io.html?highlight=export_objs#bw2io.package.BW2Package.export_objs

classmethod export_objs(objs, filename, folder='export', backwards_compatible=False)
Export a list of objects. Can have heterogeneous types.

Args:
objs (list): List of objects to export.
filename (str): Name of file to create.
folder (str, optional): Folder to create file in. Default is export.
backwards_compatible (bool, optional): Create package compatible with bw2data version 1.

Returns:
Filepath of created file.

objs should be an iterable --> list but not a dict

In [17]:
methods_with_errors_ID14=[]

for ic_name in IC_ID14_short_names:
    
    methods_with_errors_ID14.append(bw.Method(ic_name))
    
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=methods_with_errors_ID14
file_name='methods_with_errors_ID14'

bw.BW2Package.export_objs(object_name_to_save,file_name,file_path_root)

'C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\methods_with_errors_ID14.55a38b3752c9c63824403cd0633fdcc8.bw2package'

# Saving database

Saving no regular object with pickle does not work...

In [8]:
DB_for_test=bw.Database('For test')

In [44]:
act_all_EF=[act for act in DB_for_test if 'elementary' in act['name']][0]

In [51]:
len(act_all_EF.biosphere())

4029

In [7]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=DB_for_test
file_name='DB_for_test'

pickle_save(file_path_root,object_name_to_save,file_name)

Test exporting object with built-in function in BW2 : https://docs.brightwaylca.org/technical/bw2io.html?highlight=export_objs#bw2io.package.BW2Package.export_objs

classmethod export_objs(objs, filename, folder='export', backwards_compatible=False)
Export a list of objects. Can have heterogeneous types.

Args:
objs (list): List of objects to export.
filename (str): Name of file to create.
folder (str, optional): Folder to create file in. Default is export.
backwards_compatible (bool, optional): Create package compatible with bw2data version 1.

Returns:
Filepath of created file.



classmethod import_file(filepath, whitelist=True)
Import bw2package file, and create the loaded objects, including registering, writing, and processing the created objects.

Args:
filepath (str): Path of file to import
whitelist (bool): Apply whitelist to allowed types. Default is True.
Returns:
Created object or list of created objects.

In [63]:
bw.BW2Package.export_objs([DB_for_test],'DB_all_elementary_flows_test_export',r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved')

'C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\DB_all_elementary_flows_test_export.8abeee8f96a889ce46ea0d4c1ec2e863.bw2package'

Test for importing DB from another project

In [65]:
input_filepath='C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\DB_test-for-export.8cf01e09eb138502b3d2248afc2de82c.bw2package'

DB_for_test_from_import=bw.BW2Package.import_file(input_filepath)


Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 08/17/2017 18:37:58
  Finished: 08/17/2017 18:38:00
  Total time elapsed: 00:00:02
  CPU %: 34.100000
  Memory %: 1.862405


In [66]:
bw.databases

Databases dictionary with 5 object(s):
	For test
	For test_test for export
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.3 cutoff without uncertainty

In [18]:
DB_for_test_for_export=bw.Database('For test_test for export')

In [20]:
len(DB_for_test_for_export.load())

1

In [21]:
DB_for_test_for_export.copy('For test_v2')

Writing activities to SQLite3 database:
0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 08/18/2017 11:27:38
  Finished: 08/18/2017 11:27:40
  Total time elapsed: 00:00:01
  CPU %: 31.400000
  Memory %: 1.487053


Brightway2 SQLiteBackend: For test_v2

In [24]:
DB_for_test_for_export=bw.Database('For test_v2')
DB_for_test_for_export.load()

{('For test_v2', 'All elementary flows'): {'code': 'All elementary flows',
  'database': 'For test_v2',
  'exchanges': [{'amount': 1.0,
    'input': ('biosphere3', 'aa416529-bb87-40a1-bbbc-3772a7ff7848'),
    'output': ('For test_v2', 'All elementary flows'),
    'type': 'biosphere'},
   {'amount': 1.0,
    'input': ('biosphere3', '581ddcdf-82b7-4f87-83ee-761a44fea7f7'),
    'output': ('For test_v2', 'All elementary flows'),
    'type': 'biosphere'},
   {'amount': 1.0,
    'input': ('biosphere3', '2d456fb8-592b-46a7-97f0-81fc2a18e587'),
    'output': ('For test_v2', 'All elementary flows'),
    'type': 'biosphere'},
   {'amount': 1.0,
    'input': ('biosphere3', 'a6386bef-e154-4862-967c-f3767d7f9a2a'),
    'output': ('For test_v2', 'All elementary flows'),
    'type': 'biosphere'},
   {'amount': 1.0,
    'input': ('biosphere3', '0017271e-7df5-40bc-833a-36110c1fe5d5'),
    'output': ('For test_v2', 'All elementary flows'),
    'type': 'biosphere'},
   {'amount': 1.0,
    'input': ('bios

In [26]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=[DB_for_test_for_export]
file_name='DB_For test_v2'

bw.BW2Package.export_objs(object_name_to_save,file_name,file_path_root)

'C:\\bw2-python\\Notebook - tutorial\\IW+ integration\\Pickle Python object saved\\DB_For-test_v2.9536154a05725c74b8157d8b086f1aef.bw2package'

# Saving LCA object

In [56]:
DB_for_test=bw.Database('For test')

act=[act for act in DB_for_test if 'All elementary flows' in act['name']][0]
ic_name=('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017','Ecosystem Quality','Land transformation, biodiversity, GLO, with uncert')


#LCA_obj_test_export = bw.LCA({act:1}, ic_name,metadata={'module':'bw2calc.LCA.__init__','name':'bw2calc.LCA.LCA_obj_test_export'},name='LCA_obj_test_export')
LCA_obj_test_export = bw.LCA({act:1}, ic_name)
LCA_obj_test_export_2 = bw.LCA({act:1}, ic_name)

In [57]:
act.as_dict()

{'code': 'All elementary flows',
 'database': 'For test',
 'exchanges': [{'amount': 1.0,
   'input': 'Methane, bromochlorodifluoro-, Halon 1211' (kilogram, None, ('air', 'urban air close to ground')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Solids, inorganic' (kilogram, None, ('water', 'surface water')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Ethane, 1,2-dichloro-' (kilogram, None, ('water', 'surface water')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Trimethylamine' (kilogram, None, ('air', 'urban air close to ground')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Xenon-138' (kilo Becquerel, None, ('air', 'lower stratosphere + upper troposphere')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Quizalofop-p-ethyl' (kilogram, None, ('soil', 'agricultural')),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Permethrin' (kilogram, None, ('water',)),
   'type': 'biosphere'},
  {'amount': 1.0,
   'input': 'Transform

In [58]:
LCA_obj_test_export.lci()
LCA_obj_test_export.lcia()

In [75]:
LCA_obj_test_export.biosphere_matrix

<4029x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4150 stored elements in Compressed Sparse Row format>

In [76]:
LCA_obj_test_export.biosphere_dict

{('biosphere3', 'baf58fc9-573c-419c-8c16-831ac03203b9'): 1154,
 ('biosphere3', '785e9314-ccf6-4229-98c5-37881d624fc9'): 1486,
 ('biosphere3', '17f06f74-feea-4b7c-a125-a67ae6e1dcfb'): 2604,
 ('biosphere3', 'd2bf022d-9cbf-4f19-a8ec-7f507746942b'): 1434,
 ('biosphere3', 'b7a3b260-ac8d-43e2-b549-34bb5b82b533'): 20,
 ('biosphere3', '79607917-9e12-4292-b289-21ff2b36e04b'): 3016,
 ('biosphere3', '237ea455-6c82-4254-af86-9409df922094'): 1279,
 ('biosphere3', '4a1542d8-5f0e-488e-90d1-658d40c41695'): 1884,
 ('biosphere3', 'cfecebe1-1e13-458d-85bc-d40a9f934927'): 79,
 ('biosphere3', 'b93fc4df-91b2-4ac4-a315-89147515ddfb'): 3262,
 ('biosphere3', '58c8f0ec-455b-4bc1-b6fb-5d6976619b55'): 2917,
 ('biosphere3', '59570d74-fac9-446d-88a7-5c5b0a6139a7'): 2492,
 ('biosphere3', '9e253ace-bf5e-40f2-b9d3-104595b327e4'): 1159,
 ('biosphere3', '54fbc219-056c-4156-ae4f-f3a5d598665f'): 2501,
 ('biosphere3', '141ec03e-9ec3-4cfe-853d-37cc66e1c333'): 165,
 ('biosphere3', 'e91315c1-cae7-4ade-bc9a-595bfdca133f'): 102

.lci()=.load_lci_data()+.lci_calculation()

.load_lci_data() construct A abd B matrices
.lci_calculation() does the inventory calculation

Objects created be .load_lci_data can be reattributed to another LCA object :)

In [59]:
LCA_obj_test_export.bio_params

array([(1, 32621, 0, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (2, 32621, 1, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (3, 32621, 2, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False), ...,
       (4027, 32621, 4026, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (4028, 32621, 4027, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (4029, 32621, 4028, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False)], 
      dtype=[('input', '<u4'), ('output', '<u4'), ('row', '<u4'), ('col', '<u4'), ('type', 'u1'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

In [60]:
LCA_obj_test_export_2.bio_params=LCA_obj_test_export.bio_params

In [61]:
LCA_obj_test_export_2.bio_params

array([(1, 32621, 0, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (2, 32621, 1, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (3, 32621, 2, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False), ...,
       (4027, 32621, 4026, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (4028, 32621, 4027, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False),
       (4029, 32621, 4028, 0, 2, 0, 1.0, 1.0, nan, nan, nan, nan, False)], 
      dtype=[('input', '<u4'), ('output', '<u4'), ('row', '<u4'), ('col', '<u4'), ('type', 'u1'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

Test: save object created by .load_lci_data --> works with Pickle for matrices

In [62]:
techno_matrix=LCA_obj_test_export.technosphere_matrix
techno_matrix

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [10]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=[techno_matrix]
file_name='techno_matrix_test_export'

bw.BW2Package.export_objs(object_name_to_save,file_name,file_path_root)

AttributeError: metadata not found

In [63]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=techno_matrix
file_name='techno_matrix_test_export'

pickle_save(file_path_root,object_name_to_save,file_name)

In [64]:
techno_matrix_imported=pickle_load(file_path_root,file_name)

In [65]:
LCA_obj_test_export_2.technosphere_matrix=techno_matrix_imported

In [66]:
LCA_obj_test_export_2.technosphere_matrix

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

Test: save all objects created by .load_lci_data

In [67]:
LCA_obj=LCA_obj_test_export

dict_loac_lci_data_objects={'bio_params':LCA_obj.bio_params,
                           'tech_params':LCA_obj.tech_params,
                           'biosphere_dict':LCA_obj.biosphere_dict,
                           'activity_dict':LCA_obj.activity_dict,
                           'product_dict':LCA_obj.product_dict,
                           'biosphere_matrix':LCA_obj.biosphere_matrix,
                           'technosphere_matrix':LCA_obj.technosphere_matrix}

In [68]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_loac_lci_data_objects
file_name='dict_loac_lci_data_objects_test_export'

pickle_save(file_path_root,object_name_to_save,file_name)

In [69]:
dict_loac_lci_data_objects_2=pickle_load(file_path_root,file_name)

In [73]:
LCA_obj=LCA_obj_test_export_2
dict_loac_lci_data_objects=dict_loac_lci_data_objects_2

LCA_obj._fixed = True #function in .load_lci_data() --> set to True because dictionaries as already been fixed during the previous lci calculation

#to replace builder.build(self.database_filepath) in .load_lci_data()
LCA_obj.bio_params=dict_loac_lci_data_objects['bio_params']
LCA_obj.tech_params=dict_loac_lci_data_objects['tech_params']
LCA_obj.biosphere_dict=dict_loac_lci_data_objects['biosphere_dict']
LCA_obj.activity_dict=dict_loac_lci_data_objects['activity_dict']
LCA_obj.product_dict=dict_loac_lci_data_objects['product_dict']
LCA_obj.biosphere_matrix=dict_loac_lci_data_objects['biosphere_matrix']
LCA_obj.technosphere_matrix=dict_loac_lci_data_objects['technosphere_matrix']

LCA_obj_test_export_2.fix_dictionaries() #function in .load_lci_data()
LCA_obj._biosphere_dict = LCA_obj.biosphere_dict

In [74]:
#.lci() function replace by the cell below
LCA_obj.build_demand_array() #function in .lci() after calling .load_lci_data()
LCA_obj.lci_calculation() #function in .lci() after calling .load_lci_data()
LCA_obj.lcia()
LCA_obj.score


TypeError: cannot perform reduce with flexible type

In [33]:
LCA_obj_test_export_2.activity_dict

{('For test', 'All elementary flows'): 0,
 ('For test', 'Negative land transfo'): 1,
 ('For test', 'One negative land transfo'): 3,
 ('For test', 'Positive land transfo'): 2}

In [34]:
LCA_obj_test_export.activity_dict

{('For test', 'All elementary flows'): 0,
 ('For test', 'Negative land transfo'): 1,
 ('For test', 'One negative land transfo'): 3,
 ('For test', 'Positive land transfo'): 2}

A, B and C matrix available only if .lci() and .lcia() have been run

In [6]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=[LCA_obj_test_export]
file_name='LCA_obj_test_export'

bw.BW2Package.export_objs(object_name_to_save,file_name,file_path_root)

AttributeError: 'LCA' object has no attribute 'load'

Object needs to have a metadata attribute to be exported...

In [7]:
bw.projects.dir

'C:\\Users\\Laure\\AppData\\Local\\pylca\\Brightway3\\iw_integration.505e4edc32992752483974fe7e9c4c84'

In [35]:
bw.databases

Databases dictionary with 6 object(s):
	For test
	For test_test for export
	For test_v2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.3 cutoff without uncertainty

In [78]:
eiv33=bw.Database('ecoinvent 3.3 cutoff')
len(eiv33)

13831

In [79]:
13831*5000

69155000